# import libraries

In [1]:
import pandas as pd
import numpy as np
import requests

# import Beautifulsoup,scrape info from wikipedia, store data in a dataframe

In [2]:
from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


# Data cleanup and reformat

Change inteter columns to strings 

In [3]:
df.columns=df.columns.astype(str)
print(df.columns)

Index(['0', '1', '2'], dtype='object')


Change column names to assignment required ones

In [4]:
df.columns=['PostalCode','Borough','Neighborhood']


In [5]:
#drop first row of the original head from wiki site
df1=df.drop(df.index[0])
df1.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [6]:
#remove rows that does not have Borough assigned
df2=df1[df1.Borough!='Not assigned']
df2.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [13]:
#next if Neighborhood is not assigned, it will need to have same as boroug
df2['Neighborhood'][df2.Neighborhood=='Not assigned']=df2['Borough']
print(df2.head())

  PostalCode           Borough      Neighborhood
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront
6        M5A  Downtown Toronto       Regent Park
7        M6A        North York  Lawrence Heights


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


If rows have same postcode, have rows combined in Neightborhood with comma

In [14]:
df3=df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
print(df3.shape)

(103, 3)


Check data layout

In [15]:
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Check number of rows and columns in final data frame df3

In [16]:
print(df3.shape)

(103, 3)
